In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install scipy==1.15.2
!pip install datasets==2.14.4
!pip install numpy==2.0.0
!pip install sentencepiece==0.1.97
!pip install sacrebleu==2.3.1
!pip install accelerate>=0.26.0
!pip install bitsandbytes==0.41.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.6.0
    Uninstalling datasets-3.6.0:
      Successfully uninstalled datasets-3.6.0


In [3]:
!pip install gcsfs==2024.12.0
!pip install transformers==4.50.0
!pip install torch
!pip install --upgrade datasets
!pip install peft

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4


In [4]:
!pip install protobuf==3.20.3

In [5]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes --no-cache-dir
!pip install jsonlines

Found existing installation: bitsandbytes 0.41.1
Uninstalling bitsandbytes-0.41.1:
  Successfully uninstalled bitsandbytes-0.41.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 285.0 MB/s eta 0:00:00


In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
# models.py
# coding=utf-8
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Model classes for MetricX, modified from the T5 versions in HF."""

import copy
import dataclasses
from typing import Optional, Tuple, Union
import warnings

import torch
from torch import nn
import transformers
import transformers.modeling_outputs

BaseModelOutput = transformers.modeling_outputs.BaseModelOutput
ModelOutput = transformers.modeling_outputs.ModelOutput

MT5Config = transformers.models.mt5.modeling_mt5.MT5Config
MT5PreTrainedModel = transformers.models.mt5.modeling_mt5.MT5PreTrainedModel
MT5Stack = transformers.models.mt5.modeling_mt5.MT5Stack

__HEAD_MASK_WARNING_MSG = (
    transformers.models.mt5.modeling_mt5.__HEAD_MASK_WARNING_MSG  # pylint: disable=protected-access
)


@dataclasses.dataclass
class MT5ForRegressionOutput(ModelOutput):
  loss: Optional[torch.FloatTensor] = None
  predictions: torch.FloatTensor = None


class MT5ForRegression(MT5PreTrainedModel):
  """MT5 model for regression."""

  def __init__(self, config: MT5Config):
    super().__init__(config)
    self.model_dim = config.d_model

    self.shared = nn.Embedding(config.vocab_size, config.d_model)

    encoder_config = copy.deepcopy(config)
    encoder_config.is_decoder = False
    encoder_config.use_cache = False
    encoder_config.is_encoder_decoder = False
    self.encoder = MT5Stack(encoder_config, self.shared)

    decoder_config = copy.deepcopy(config)
    decoder_config.is_decoder = True
    decoder_config.is_encoder_decoder = False
    decoder_config.num_layers = config.num_decoder_layers
    self.decoder = MT5Stack(decoder_config, self.shared)

    self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

    # Initialize weights and apply final processing
    self.post_init()

    # Model parallel
    self.model_parallel = False
    self.device_map = None

  def forward(
      self,
      input_ids: Optional[torch.LongTensor] = None,
      attention_mask: Optional[torch.FloatTensor] = None,
      decoder_attention_mask: Optional[torch.BoolTensor] = None,
      head_mask: Optional[torch.FloatTensor] = None,
      decoder_head_mask: Optional[torch.FloatTensor] = None,
      cross_attn_head_mask: Optional[torch.Tensor] = None,
      encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
      past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
      inputs_embeds: Optional[torch.FloatTensor] = None,
      decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
      labels: Optional[torch.FloatTensor] = None,
      use_cache: Optional[bool] = None,
      output_attentions: Optional[bool] = None,
      output_hidden_states: Optional[bool] = None,
      return_dict: Optional[bool] = None,
  ) -> Union[Tuple[torch.FloatTensor], MT5ForRegressionOutput]:
    use_cache = use_cache if use_cache is not None else self.config.use_cache
    return_dict = (
        return_dict if return_dict is not None else self.config.use_return_dict
    )

    # FutureWarning: head_mask was separated into two input args - head_mask,
    # decoder_head_mask
    if head_mask is not None and decoder_head_mask is None:
      if self.config.num_layers == self.config.num_decoder_layers:
        warnings.warn(__HEAD_MASK_WARNING_MSG, FutureWarning)
        decoder_head_mask = head_mask

    # Encode if needed (training, first prediction pass)
    if encoder_outputs is None:
      # Convert encoder inputs in embeddings if needed
      encoder_outputs = self.encoder(
          input_ids=input_ids,
          attention_mask=attention_mask,
          inputs_embeds=inputs_embeds,
          head_mask=head_mask,
          output_attentions=output_attentions,
          output_hidden_states=output_hidden_states,
          return_dict=return_dict,
      )
    elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
      encoder_outputs = BaseModelOutput(
          last_hidden_state=encoder_outputs[0],
          hidden_states=encoder_outputs[1]
          if len(encoder_outputs) > 1
          else None,
          attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
      )

    hidden_states = encoder_outputs[0]

    if self.model_parallel:
      torch.cuda.set_device(self.decoder.first_device)

    # Create 1 step of dummy input for the decoder.
    batch_size = input_ids.size(0)
    decoder_input_ids = torch.LongTensor([0]).repeat(batch_size).reshape(-1, 1)
    if torch.cuda.is_available():
      decoder_input_ids = decoder_input_ids.to(torch.device("cuda"))

    # Set device for model parallelism
    if self.model_parallel:
      torch.cuda.set_device(self.decoder.first_device)
      hidden_states = hidden_states.to(self.decoder.first_device)
      if decoder_input_ids is not None:
        decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
      if attention_mask is not None:
        attention_mask = attention_mask.to(self.decoder.first_device)
      if decoder_attention_mask is not None:
        decoder_attention_mask = decoder_attention_mask.to(
            self.decoder.first_device
        )

    # Decode
    decoder_outputs = self.decoder(
        input_ids=decoder_input_ids,
        attention_mask=decoder_attention_mask,
        inputs_embeds=decoder_inputs_embeds,
        past_key_values=past_key_values,
        encoder_hidden_states=hidden_states,
        encoder_attention_mask=attention_mask,
        head_mask=decoder_head_mask,
        cross_attn_head_mask=cross_attn_head_mask,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    sequence_output = decoder_outputs[0]

    # Set device for model parallelism
    if self.model_parallel:
      torch.cuda.set_device(self.encoder.first_device)
      self.lm_head = self.lm_head.to(self.encoder.first_device)
      sequence_output = sequence_output.to(self.lm_head.weight.device)

    if self.config.tie_word_embeddings:
      # Rescale output before projecting on vocab
      # See
      # https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/transformer/transformer.py#L586
      sequence_output = sequence_output * (self.model_dim**-0.5)

    lm_logits = self.lm_head(sequence_output)

    # 250089 = <extra_id_10>
    predictions = lm_logits[:, 0, 250089]

    # Clip to 0 to 25
    predictions = torch.clamp(predictions, 0, 25)

    loss = None
    if labels is not None:
      loss_fct = nn.MSELoss()
      # move labels to correct device to enable PP
      labels = labels.to(predictions.device)
      loss = loss_fct(predictions.view(-1), labels.view(-1))

    return MT5ForRegressionOutput(
        loss=loss,
        predictions=predictions,
    )



In [ ]:
# predict.py
# coding=utf-8
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Runs inference with a MetricX model."""

import dataclasses
import json
import os

import datasets
import torch
import transformers
import sys
from io import StringIO
import argparse
import os
os.environ["WANDB_DISABLED"] = "true"

# Save the original command line arguments
old_argv = sys.argv

# Set up fake command line arguments for predict.py
input_file = "/content/drive/MyDrive/translation_outputs/flores-contrastive/flores/ig-en/input_source.jsonl"
output_file = "/content/drive/MyDrive/translation_outputs/flores-contrastive/flores/ig-en/output_source.jsonl"

sys.argv = [
    'predict.py',
    '--tokenizer', 'google/mt5-xl',
    '--model_name_or_path', 'google/metricx-23-xl-v2p0',
    '--max_input_length', '1200',
    '--batch_size', '1',
    '--input_file', input_file,
    '--output_file', output_file
]

# Capture any output (optional)
output = StringIO()

@dataclasses.dataclass
class Arguments:
  """Prediction command-line arguments."""

  tokenizer: str = dataclasses.field(
      metadata={"help": "The name of the tokenizer"},
  )

  model_name_or_path: str = dataclasses.field(
      metadata={
          "help": (
              "Path to pretrained model or model identifier from"
              " huggingface.co/models"
          )
      },
  )

  max_input_length: int = dataclasses.field(
      metadata={"help": "The maximum allowable input sequence length."},
  )

  batch_size: int = dataclasses.field(
      metadata={"help": "The global prediction batch size."},
  )

  input_file: str = dataclasses.field(metadata={"help": "The input file."})

  output_file: str = dataclasses.field(
      metadata={"help": "The output file with predictions."},
  )

  qe: bool = dataclasses.field(
      metadata={"help": "Indicates the metric is a QE metric."},
      default=False,
  )


def get_dataset(
    input_file: str, tokenizer, max_input_length: int, device, is_qe: bool
):
  """Gets the test dataset for prediction.

  If `is_qe` is true, the input data must have "hypothesis" and "source" fields.
  If it is false, there must be "hypothesis" and "reference" fields.

  Args:
    input_file: The path to the jsonl input file.
    tokenizer: The tokenizer to use.
    max_input_length: The maximum input sequence length.
    device: The ID of the device to put the PyTorch tensors on.
    is_qe: Indicates whether the metric is a QE metric or not.

  Returns:
    The dataset.
  """

  def _make_input(example):
    if is_qe:
      example["input"] = (
          "candidate: "
          + example["hypothesis"]
          + " source: "
          + example["source"]
      )
    else:
      example["input"] = (
          "candidate: "
          + example["hypothesis"]
          + " reference: "
          + example["reference"]
      )
    return example


  def _tokenize(example):
    return tokenizer(
        example["input"],
        max_length=max_input_length,
        truncation=True,
        padding=False,
    )


  def _remove_eos(example):
    example["input_ids"] = example["input_ids"][:-1]
    example["attention_mask"] = example["attention_mask"][:-1]
    return example

  ds = datasets.load_dataset("json", data_files={"test": input_file})
  ds = ds.map(_make_input)
  ds = ds.map(_tokenize)
  ds = ds.map(_remove_eos)
  ds.set_format(
      type="torch",
      columns=["input_ids", "attention_mask"],
      device=device,
      output_all_columns=True,
  )
  return ds


def main() -> None:
  parser = transformers.HfArgumentParser(Arguments)
  (args,) = parser.parse_args_into_dataclasses()

  if torch.cuda.is_available():
    device = torch.device("cuda")
    per_device_batch_size = args.batch_size // torch.cuda.device_count()
  else:
    device = torch.device("cpu")
    per_device_batch_size = args.batch_size

  tokenizer = transformers.AutoTokenizer.from_pretrained(args.tokenizer)

  model = MT5ForRegression.from_pretrained(args.model_name_or_path)
  model.to(device)
  model.eval()


  ds = get_dataset(
      args.input_file,
      tokenizer,
      args.max_input_length,
      device,
      args.qe,
  )


  training_args = transformers.TrainingArguments(
      output_dir=os.path.dirname(args.output_file),
      per_device_eval_batch_size=per_device_batch_size,
      dataloader_pin_memory=False,
      report_to="none"
  )
  trainer = transformers.Trainer(
      model=model,
      args=training_args,
  )
  predictions, _, _ = trainer.predict(test_dataset=ds["test"])

  dirname = os.path.dirname(args.output_file)
  if dirname:
    os.makedirs(dirname, exist_ok=True)

  with open(args.output_file, "w") as out:
    for pred, example in zip(predictions, ds["test"]):
      example["prediction"] = float(pred)
      del example["input"]
      del example["input_ids"]
      del example["attention_mask"]
      out.write(json.dumps(example) + "\n")


if __name__ == "__main__":
  main()

# Restore original command line arguments
sys.argv = old_argv

# Show output if needed
print(output.getvalue())

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# Load the three result files

import jsonlines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Configuration
FILES = {
    "Baseline": "/content/drive/MyDrive/translation_outputs/flores-contrastive/omissions/flores/ig-en/output_direct.jsonl",
    "SystemPrompt": "/content/drive/MyDrive/translation_outputs/flores-contrastive/omissions/flores/ig-en/output_contrastive_system.jsonl",
    "UserMessage": "/content/drive/MyDrive/translation_outputs/flores-contrastive/omissions/flores/ig-en/output_contrastive_user.jsonl"
}

def load_and_analyze():
    data = {}
    lengths = set()
    for name, path in FILES.items():
        with jsonlines.open(path) as rdr:
            scores = np.array([ex["prediction"] for ex in rdr])
            data[name] = scores
            lengths.add(len(scores))
    if len(lengths) > 1:
        raise ValueError(f"File length mismatch: {lengths}")

    # Create global variables for compatibility with existing code
    global base, user, system
    base = data["Baseline"]
    user = data["UserMessage"]
    system = data["SystemPrompt"]

    return data

print("🔄 Loading data...")
data = load_and_analyze()

def analyze_results(data):
    metrics = []
    for name, scores in data.items():
        metrics.append({
            "Method": name,
            "Mean": scores.mean(),
            "Median": np.median(scores),
            "Std": scores.std(),
            "Min": scores.min(),
            "Max": scores.max(),
            "Q1": np.percentile(scores, 25),
            "Q3": np.percentile(scores, 75),
            "IQR": np.percentile(scores, 75) - np.percentile(scores, 25),
            "Skewness": stats.skew(scores),
            "Kurtosis": stats.kurtosis(scores)
        })
    return pd.DataFrame(metrics).set_index("Method").style.format("{:.2f}")

print("\n📊 Statistical Summary:")
display(analyze_results(data))

🔄 Loading data...

📊 Statistical Summary:


,Mean,Median,Std,Min,Max,Q1,Q3,IQR,Skewness,Kurtosis
Method,,,,,,,,,,
Baseline,5.97,5.23,3.93,0.00,23.60,2.98,7.95,4.97,1.15,1.47
SystemPrompt,5.79,4.99,3.80,0.00,24.17,2.99,7.51,4.52,1.32,2.39
UserMessage,5.77,5.15,3.86,0.00,25.00,2.93,7.49,4.56,1.33,2.53


In [9]:
def delta_report(new_scores, baseline_scores, name):
    deltas = new_scores - baseline_scores
    improved = np.sum(deltas < 0)
    worsened = np.sum(deltas > 0)
    unchanged = np.sum(deltas == 0)

    print(f"\n{name} Comparison:")
    print(f"→ Improved segments: {improved:4d} ({improved/len(deltas):.1%})")
    print(f"→ Worsened segments: {worsened:4d} ({worsened/len(deltas):.1%})")
    print(f"→ Unchanged segments: {unchanged:4d}")
    print(f"→ Mean Δ: {deltas.mean():+.3f} | Median Δ: {np.median(deltas):+.3f}")
    return deltas

print("\n=== Segment-Level Improvement Analysis ===")
user_deltas = delta_report(user, base, "UserMessage vs Baseline")
system_deltas = delta_report(system, base, "SystemPrompt vs Baseline")


=== Segment-Level Improvement Analysis ===

UserMessage vs Baseline Comparison:
→ Improved segments:  526 (52.0%)
→ Worsened segments:  451 (44.6%)
→ Unchanged segments:   35
→ Mean Δ: -0.206 | Median Δ: -0.068

SystemPrompt vs Baseline Comparison:
→ Improved segments:  496 (49.0%)
→ Worsened segments:  450 (44.5%)
→ Unchanged segments:   66
→ Mean Δ: -0.182 | Median Δ: +0.000


In [10]:
print("\n=== Quality Tier Analysis (Lower = Better) ===")
quality_tiers = {
    "Excellent (0-5)": (0, 5),
    "Good (5-10)": (5, 10),
    "Fair (10-15)": (10, 15),
    "Poor (15-20)": (15, 20),
    "Very Poor (20-25)": (20, 25)
}

tier_df = pd.DataFrame()
for name, (low, high) in quality_tiers.items():
    base_mask = (base >= low) & (base < high)
    user_mask = (user >= low) & (user < high)
    system_mask = (system >= low) & (system< high)

    tier_df.loc[name, 'Baseline'] = f"{base_mask.sum():d} ({base_mask.mean():.1%})"
    tier_df.loc[name, 'UserMessage'] = f"{user_mask.sum():d} ({user_mask.mean():.1%})"
    tier_df.loc[name, 'SystemPrompt'] = f"{system_mask.sum():d} ({system_mask.mean():.1%})"
    tier_df.loc[name, 'UserMessage Δ'] = f"{(user_mask.mean()-base_mask.mean()):+.1%}"
    tier_df.loc[name, 'SystemPrompt Δ'] = f"{(system_mask.mean()-base_mask.mean()):+.1%}"

def color_negative_green(val):
    color = 'green' if '-' in val and val!='+0.0%' else 'red' if '+' in val else 'black'
    return f'color: {color}'

display(tier_df.style.applymap(color_negative_green, subset=['UserMessage Δ', 'SystemPrompt Δ']))


=== Quality Tier Analysis (Lower = Better) ===


<ipython-input-10-504e64140901>:26: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(tier_df.style.applymap(color_negative_green, subset=['UserMessage Δ', 'SystemPrompt Δ']))


,Baseline,UserMessage,SystemPrompt,UserMessage Δ,SystemPrompt Δ
Excellent (0-5),481 (47.5%),482 (47.6%),507 (50.1%),+0.1%,+2.6%
Good (5-10),384 (37.9%),404 (39.9%),375 (37.1%),+2.0%,-0.9%
Fair (10-15),111 (11.0%),93 (9.2%),101 (10.0%),-1.8%,-1.0%
Poor (15-20),31 (3.1%),27 (2.7%),22 (2.2%),-0.4%,-0.9%
Very Poor (20-25),5 (0.5%),4 (0.4%),7 (0.7%),-0.1%,+0.2%
